## 4.6 Merging and Exporting Data part 2

### This script contains the following points:

#### 1. Create data to experiment on
#### 2. Merge data frames
#### 3. Export data frame

In [1]:
##import libraries
import pandas as pd
import numpy as np
import os

In [2]:
## 3: import orders_products_combined
path = r'/Users/rachelallen/CareerFoundry Achievement 4/08-2022 Instacart Basket Analysis/02 Data'

In [3]:
df_orders_products_combined = pd.read_pickle(os.path.join(path, 'Prepared Data', 'orders_products_combined.pkl'))

In [4]:
## 4: check the shape of imported dataframe
df_orders_products_combined.shape

(32434489, 11)

In [5]:
df_orders_products_combined.head()

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,flagged,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,No Previous Order,196,1,0,both
1,2539329,1,1,2,8,NaN,No Previous Order,14084,2,0,both
2,2539329,1,1,2,8,NaN,No Previous Order,12427,3,0,both
3,2539329,1,1,2,8,NaN,No Previous Order,26088,4,0,both
4,2539329,1,1,2,8,NaN,No Previous Order,26405,5,0,both


In [6]:
##import cleaned products dataset
df_prods = pd.read_csv(os.path.join(path, 'Prepared Data', 'products_prepared.csv'))

In [7]:
df_prods.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices
0,0,1,Chocolate Sandwich Cookies,61,19,5.8
1,1,2,All-Seasons Salt,104,13,9.3
2,2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,4,5,Green Chile Anytime Sauce,5,13,4.3


In [8]:
##whoops what the heck lets drop that unnamed 0 column
df_prods = df_prods.drop(columns = ['Unnamed: 0'])

In [9]:
df_prods.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [10]:
## determine a suitable way to combine the dfs
df_prods.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices'], dtype='object')

In [11]:

df_orders_products_combined.columns

Index(['order_id', 'user_id', 'order_number', 'order_day_of_week',
       'order_hour', 'days_since_prior_order', 'flagged', 'product_id',
       'add_to_cart_order', 'reordered', '_merge'],
      dtype='object')

Looks like the column they have in common is product_id. I don't know what information I can add or drop so a full join seems the most appropriate.

In [12]:
## merge frames
df_merged = df_orders_products_combined.merge(df_prods, on = ['product_id'], indicator = 'Exists')

In [13]:
df_merged.shape

(32404859, 16)

okay whoops I lost some rows there, let me try using an outer join

In [14]:
## merge frames
df_merged2 = df_orders_products_combined.merge(df_prods, on = ['product_id'], indicator = 'Exists', how = 'outer')

In [15]:
df_merged2.shape

(32435070, 16)

In [16]:
# now the number has gone UP

In [17]:
df_merged2.head()

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,flagged,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,Exists
0,2539329.0,1.0,1.0,2.0,8.0,NaN,No Previous Order,196,1.0,0.0,both,Soda,77.0,7.0,9.0,both
1,2398795.0,1.0,2.0,3.0,7.0,15.0,Previous Order,196,1.0,1.0,both,Soda,77.0,7.0,9.0,both
2,473747.0,1.0,3.0,3.0,12.0,21.0,Previous Order,196,1.0,1.0,both,Soda,77.0,7.0,9.0,both
3,2254736.0,1.0,4.0,4.0,7.0,29.0,Previous Order,196,1.0,1.0,both,Soda,77.0,7.0,9.0,both
4,431534.0,1.0,5.0,4.0,15.0,28.0,Previous Order,196,1.0,1.0,both,Soda,77.0,7.0,9.0,both


I was getting an error reading: "ValueError: Cannot use name of an existing column for indicator column" so I checked Stack Overflow and it turns out that since my first merge had created a '_merge'_ column I couldn't create another one. They suggested changing the indicator = True to indicator = Exists so that's what I did. 

In [18]:
df_merged2['Exists'].value_counts()

both          32404859
left_only        30200
right_only          11
Name: Exists, dtype: int64

In [19]:
## 6: confirm using the merge flag 
## test to see if this actually worked by using how = outer
df_merged_checked = df_orders_products_combined.merge(df_prods, on = ['product_id'], indicator = 'Exists', how = 'outer')

In [20]:
df_merged_checked['Exists'].value_counts()

both          32404859
left_only        30200
right_only          11
Name: Exists, dtype: int64

Good thing I checked to see the left_only and right_only values -- the task directions don't indicate I should do anything about this but it's good to know.

In [19]:
df_merged2.shape

(32435070, 16)

In [23]:
df_prods.shape

(49672, 5)

In [22]:
32434489 - 32404859

29630

In [24]:
##looks like I am about 29630 rows shorter than I was when I started -- where did they go?
## update I went back and did an outer join

In [20]:
## 7 export as pkl file as orders_products_merged
df_merged2.to_pickle(os.path.join(path, 'Prepared Data', 'orders_products_merged2.pkl' ))